## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
import requests

In [3]:
import gmaps

In [4]:
# Import API key
from config import g_key

In [5]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
# I made a copy of the weather_database_csv folder inside the Vacation_Search folder to access it
#otherwise Filenot found error.
city_data_df = pd.read_csv("weather_database_csv/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Noumea,NC,-22.28,166.46,86.00,55,75,5.82,broken clouds
1,1,Punta Arenas,CL,-53.15,-70.92,57.20,67,0,16.11,clear sky
2,2,Laguna,US,38.42,-121.42,63.00,38,1,5.82,clear sky
3,3,Busselton,AU,-33.65,115.33,57.99,71,49,1.01,scattered clouds
4,4,Sitka,US,57.05,-135.33,50.00,81,90,9.17,light rain


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Noumea,NC,-22.28,166.46,86.00,55,75,5.82,broken clouds
2,2,Laguna,US,38.42,-121.42,63.00,38,1,5.82,clear sky
5,5,Kavieng,PG,-2.57,150.80,84.61,68,0,11.34,clear sky
7,7,Kapaa,US,22.08,-159.32,82.40,65,1,8.05,clear sky
9,9,Hermanus,ZA,-34.42,19.23,64.00,83,94,5.99,overcast clouds
12,12,Sao Joao Da Barra,BR,-21.64,-41.05,75.96,90,97,8.72,overcast clouds
13,13,Metro,ID,-5.11,105.31,76.48,89,87,10.63,overcast clouds
18,18,Qandala,SO,11.47,49.87,76.26,62,30,1.81,scattered clouds
19,19,Afonso Bezerra,BR,-5.50,-36.51,77.54,66,1,21.23,clear sky
25,25,Abu Dhabi,AE,24.47,54.37,73.40,88,0,2.24,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# This is for me to confirm the number of cities without null values.
clean_df.notnull().sum()

City_ID                323
City                   323
Country                321
Lat                    323
Lng                    323
Max Temp               323
Humidity               323
Cloudiness             323
Wind Speed             323
Current Description    323
dtype: int64

In [11]:
# This is for me to confirm that the number of cities without null values equals the toal number of cities
# which means cell 6 is correct no null values.
clean_df.count()

City_ID                323
City                   323
Country                321
Lat                    323
Lng                    323
Max Temp               323
Humidity               323
Cloudiness             323
Wind Speed             323
Current Description    323
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [13]:
clean_df.notnull().sum()

City_ID                321
City                   321
Country                321
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Current Description    321
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Noumea,NC,86.00,broken clouds,-22.28,166.46,
2,Laguna,US,63.00,clear sky,38.42,-121.42,
5,Kavieng,PG,84.61,clear sky,-2.57,150.80,
7,Kapaa,US,82.40,clear sky,22.08,-159.32,
9,Hermanus,ZA,64.00,overcast clouds,-34.42,19.23,
12,Sao Joao Da Barra,BR,75.96,overcast clouds,-21.64,-41.05,
13,Metro,ID,76.48,overcast clouds,-5.11,105.31,
18,Qandala,SO,76.26,scattered clouds,11.47,49.87,
19,Afonso Bezerra,BR,77.54,clear sky,-5.50,-36.51,
25,Abu Dhabi,AE,73.40,clear sky,24.47,54.37,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 50000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [18]:
# 8a. Create the output File (CSV)
# Saved the WeatherPy_vacation.csv inside the weather_database_csv folder.
output_data_file = "weather_database_csv/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))